In [1]:
import pathlib

import pandas as pd

#dataset_full_path = pathlib.Path('/data/road_surface_classifier/rsc_naip_L17c/dataset_simple/dataset_full.csv')
features_path = pathlib.Path('/data/road_surface_classifier/features.csv')

#df_dataset = pd.read_csv(dataset_full_path).iloc[:, 1:].set_index('osm_id')
df_feat = pd.read_csv(features_path).iloc[:, 1:].set_index('osm_id')

In [3]:
df_combined = df_feat # df_dataset.merge(df_feat, how='left', on='osm_id')
# df_combined = df_combined.drop(columns=['class_num', 'highway_num', 'wkt', 'x', 'y', 'ix', 'iy', 'length', 'dist_sq'])
df_combined = df_combined.drop(columns=['wkt', 'x', 'y', 'ix', 'iy', 'length'])
df_combined

,highway,surface,lon,lat
osm_id,,,,
391020748,primary,asphalt,-117.658360,33.596795
14290622,primary_link,asphalt,-115.147473,36.092396
240534097,motorway,asphalt,-78.739247,42.929866
684838122,secondary,asphalt,-112.012969,33.444280
13567313,unclassified,asphalt,-88.843204,32.081660
...,...,...,...,...
13853468,unclassified,unpaved,-117.039719,47.376109
14121032,unclassified,unpaved,-98.947994,42.185050
8757986,residential,unpaved,-74.787238,44.285550


In [26]:
from osgeo import ogr, osr

# Create SRS (EPSG:4326: WGS-84 decimal degrees)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)

driver = ogr.GetDriverByName('GPKG')
ds: ogr.DataSource = driver.CreateDataSource('/data/road_surface_classifier/features.gpkg')
layer: ogr.Layer = ds.CreateLayer('data', srs=srs, geom_type=ogr.wkbPoint)

osm_id_field = ogr.FieldDefn('osm_id', ogr.OFTInteger64)
highway_field = ogr.FieldDefn('highway', ogr.OFTString)
surface_field = ogr.FieldDefn('surface', ogr.OFTString)

layer.CreateField(osm_id_field)
layer.CreateField(highway_field)
layer.CreateField(surface_field)

feature_defn = layer.GetLayerDefn()

for _, row in df_combined.iterrows():
    feat = ogr.Feature(feature_defn)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint_2D(row['lon'], row['lat'])

    feat.SetGeometry(pt)
    feat.SetField('osm_id', row.name)
    feat.SetField('highway', row['highway'])
    feat.SetField('surface', row['surface'])
    layer.CreateFeature(feat)
    pt = None
    feat = None

layer = None # type: ignore
ds = None # type: ignore